In [1]:
import pandas as pd
import refinitiv.data as rd  # This is LSEG's Data and Analytics' API wrapper, called the Refinitiv Data Library for Python.
from refinitiv.data.content import historical_pricing  # We will use this Python Class in `rd` to show the Implied Volatility data already available before our work.
import refinitiv.data.content.ipa.financial_contracts as rdf  # We're going to need thtis to use the content layer of the RD library and the calculators of greeks and Impl Volat in Instrument Pricing Analytics (IPA) and Exchange Traded Instruments (ETI)
from refinitiv.data.content.ipa.financial_contracts import option  # We're going to need thtis to use the content layer of the RD library and the calculators of greeks and Impl Volat in IPA & ETI

# Let's authenticate ourseves to LSEG's Data and Analytics service, Refinitiv:
try:  # The following libraries are not available in Codebook, thus this try loop
    rd.get_config().set_param(
        param=f"logs.transports.console.enabled", value=True)
    rd.open_session(config_name="C:\\Example.DataLibrary.Python-main\\Configuration\\refinitiv-data.config.json")
    session = rd.open_session("desktop.workspace")
    session.set_log_level("DEBUG")
except:
    rd.open_session()

c:\Users\U6082174\AppData\Local\Programs\Python\Python311\Lib\site-packages\requests\__init__.py:109:RequestsDependencyWarning: urllib3 (2.2.0) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!


In [2]:
live_inst = '2HSI16200L8.HF'  # '2HSI16200L8.HF'

In [3]:
rd.get_history(live_inst,
    interval='1D',
    fields=['SETTLE'],
    start='2023-10-01',
    end='2024-01-09')

2HSI16200L8.HF,SETTLE
Date,
2023-10-03,4833
2023-10-04,4804
2023-10-05,4772
2023-10-06,4974
2023-10-09,5016
...,...
2024-01-03,3785
2024-01-04,3770
2024-01-05,3721


In [4]:
HSI_test0 = rd.content.historical_pricing.summaries.Definition(
    live_inst,
    interval=rd.content.historical_pricing.Intervals.DAILY,
    fields=['SETTLE'],
    start='2023-10-01',
    end='2024-01-09').get_data().data.df

In [5]:
HSI_test0

2HSI16200L8.HF,SETTLE
Date,
2023-10-03,4833
2023-10-04,4804
2023-10-05,4772
2023-10-06,4974
2023-10-09,5016
...,...
2024-01-03,3785
2024-01-04,3770
2024-01-05,3721


In [6]:
HSI_test0.index[-1].strftime('%Y-%m-%d')

'2024-01-09'

In [7]:
hk_rf = 100 - rd.get_history(
    universe=['HK3MT=RR'],  # HK10YGB=EODF, HKGOV3MZ=R, HK3MT=RR
    fields=['TR.MIDPRICE'],
    start=HSI_test0.index[0].strftime('%Y-%m-%d'),
    end=HSI_test0.index[-1].strftime('%Y-%m-%d'))

In [8]:
HSI_test1 = pd.merge(
    HSI_test0, hk_rf, left_index=True, right_index=True)

In [9]:
HSI_test1 = HSI_test1.rename(
    columns={"SETTLE": "OptionPrice", "Mid Price": "RfRatePrct"})

In [10]:
hist_HSI_undrlying_pr = rd.get_history(
    universe=['.HSI'],
    fields=["TRDPRC_1"],
    # interval="1D",
    start=HSI_test0.index[0].strftime('%Y-%m-%d'),
    end=HSI_test0.index[-1].strftime('%Y-%m-%d'))

In [11]:
HSI_test2 = pd.merge(
    HSI_test1, hist_HSI_undrlying_pr,
    left_index=True, right_index=True)

In [12]:
HSI_test2 = HSI_test2.rename(
    columns={"TRDPRC_1": "UndrlyingPr"})

In [13]:
HSI_test2.columns.name = live_inst
HSI_test2

2HSI16200L8.HF,OptionPrice,RfRatePrct,UndrlyingPr
Date,,,
2023-10-04,4804,1.146,17195.84
2023-10-05,4772,1.138,17213.87
2023-10-06,4974,1.095,17485.98
2023-10-09,5016,1.094,17517.4
2023-10-10,5235,1.1715,17664.73
...,...,...,...
2024-01-03,3785,1.088,16646.41
2024-01-04,3770,1.086,16645.98
2024-01-05,3721,1.038,16535.33


In [14]:
request_fields = ['Strike', 'MarketValueInDealCcy', 'RiskFreeRatePercent', 'UnderlyingPrice', 'PricingModelType', 'DividendType', 'UnderlyingTimeStamp', 'ReportCcy', 'VolatilityType', 'Volatility', 'DeltaPercent', 'GammaPercent', 'RhoPercent', 'ThetaPercent', 'VegaPercent', 'ErrorMessage']

# Content Layer

In [15]:
response = option.Definition()

c:\Users\U6082174\AppData\Local\Programs\Python\Python311\Lib\site-packages\refinitiv\data\content\ipa\financial_contracts\option\_definition.py:177:FutureWarning: The rd.content.ipa.financial_contracts.option module will be removed in future library version v2.0. Please use rd.content.ipa.financial_contracts.fx_option or rd.content.ipa.financial_contracts.eti_option instead.


In [16]:
response = option.Definition(
    # asian_definition: EtiFixingInfo | FxAverageInfo = None,
    # barrier_definition: FxBarrierDefinition | EtiBarrierDefinition = None,
    # binary_definition: FxBinaryDefinition | EtiBinaryDefinition = None,
    buy_sell=option.BuySell.BUY,
    call_put=option.CallPut.CALL,
    # cbbc_definition: EtiCbbcDefinition | None = None,  # Call Bear/Bull Contract
    # deal_contract: int | None = None,  # The number of contracts bought or sold in the deal.
    # delivery_date: OptDateTime = None,
    # double_barrier_definition: FxDoubleBarrierDefinition | None = None,
    # double_barriers_definition: EtiDoubleBarriersDefinition | None = None,
    # double_binary_definition: FxDoubleBinaryDefinition = None,
    # dual_currency_definition: FxDualCurrencyDefinition | None = None,
    # end_date: OptDateTime = None,  # The maturity or expiry date of the instrument. The value is expressed in ISO 8601 format: YYYY-MM-DDT[hh]:[mm]:[ss]Z (e.g., '2021-01-01T00:00:00Z'). Mandatory for OTC ETI options and FX options (if tenor is not defined).
    # end_date_time: OptDateTime = None,
    # exercise_style: ExerciseStyle | str = None,
    # forward_start_definition: FxForwardStart | None = None,
    instrument_code=live_inst,
    # instrument_tag: str | None = None,
    # lot_size: float | None = None,
    # notional_amount: float | None = None,
    # notional_ccy="HKD",
    # payments: InputFlow | None = None,
    # settlement_ccy="HKD",
    # settlement_type: SettlementType | str = None,
    # start_date: OptDateTime = None,
    strike=float(4200),
    # tenor: str | None = None,
    # time_zone_offset: int | None = None,
    underlying_definition=option.EtiUnderlyingDefinition(instrument_code=live_inst),
    underlying_type=option.UnderlyingType.ETI,
    fields=request_fields,
    pricing_parameters=option.PricingParameters(
        # atm_volatility_object: BidAskMid | None = None,
        # butterfly10_d_object: BidAskMid | None = None,
        # butterfly25_d_object: BidAskMid | None = None,
        # domestic_deposit_rate_percent_object: BidAskMid | None = None,
        # foreign_deposit_rate_percent_object: BidAskMid | None = None,
        # forward_points_object: BidAskMid | None = None,
        # fx_spot_object: BidAskMid | None = None,
        # fx_swap_calculation_method: FxSwapCalculationMethod | str = None,
        # implied_volatility_object: BidAskMid | None = None,
        # interpolation_weight: InterpolationWeight | None = None,
        # option_price_side: PriceSide | str = None,
        # option_time_stamp: TimeStamp | str = None,
        # payout_custom_dates: List[str] | None = None,
        # payout_scaling_interval: PayoutScaling | None = None,
        # price_side: PriceSide | str = None,
        pricing_model_type='BlackScholes',
        # risk_reversal10_d_object: BidAskMid | None = None,
        # risk_reversal25_d_object: BidAskMid | None = None,
        # underlying_price_side: PriceSide | str = None,
        # underlying_time_stamp: TimeStamp | None = None,
        # volatility_model: VolatilityModel | str = None,
        volatility_type="Implied",
        # compute_payout_chart: bool | None = None,
        # compute_volatility_payout: bool | None = None,
        # cutoff_time: str | None = None,
        # cutoff_time_zone: str | None = None,
        # market_data_date: OptDateTime = None,
        market_value_in_deal_ccy=float(HSI_test2['OptionPrice'][0]),
        # market_value_in_report_ccy: float | None = None,
        report_ccy="HKD",
        # report_ccy_rate: float | None = None,
        risk_free_rate_percent=float(HSI_test2['RfRatePrct'][0]),
        # simulate_exercise: bool | None = None,
        underlying_price=float(HSI_test2['UndrlyingPr'][0]),
        valuation_date=HSI_test0.index[0].strftime('%Y-%m-%d'),
        # volatility: float | None = None,
        # volatility_percent: float | None = None)
    # extended_params: ExtendedParams = None
    )).get_data()

RDError: Error code None | Unable to calculate the Implied Volatility.

In [17]:
live_universe = [
        {
          "instrumentType": "Option",
          "instrumentDefinition": {
            "buySell": "buy",
            "underlyingType": "Eti",
            "instrumentCode": live_inst,
          },
          "pricingParameters": {
            "marketValueInDealCcy": float(HSI_test2['OptionPrice'][i]),
            "riskFreeRatePercent": float(HSI_test2['RfRatePrct'][i]),
            "underlyingPrice": float(HSI_test2['UndrlyingPr'][i]),
            "pricingModelType": "BlackScholes",
            "dividendType": "ImpliedYield",
            "volatilityType": "Implied",
            "underlyingTimeStamp": "Default",
            "reportCcy": "HKD"
          }
        }
      for i in range(len(HSI_test2.index))]

In [18]:
def Chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [19]:
batchOf = 100  # 100 is the max
for i, j in enumerate(Chunks(live_universe, 100)):
    print(f"Batch of {batchOf} requests no. {str(i+1)}/{str(len([i for i in Chunks(live_universe, batchOf)]))} started")
    # Example request with Body Parameter - Symbology Lookup
    live_troubleshoot_request_definition = rd.delivery.endpoint_request.Definition(
        method=rd.delivery.endpoint_request.RequestMethod.POST,
        url='https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts',
        body_parameters={"fields": request_fields,
                         "outputs": ["Data", "Headers"],
                         "universe": j})

    live_troubleshoot_resp = live_troubleshoot_request_definition.get_data()
    headers_name = [h['name'] for h in live_troubleshoot_resp.data.raw['headers']]

    if i == 0:
        live_troubleshoot_df = pd.DataFrame(
            data=live_troubleshoot_resp.data.raw['data'],
            columns=headers_name)
    else:
        _live_troubleshoot_df = pd.DataFrame(
            data=live_troubleshoot_resp.data.raw['data'],
            columns=headers_name)
        live_troubleshoot_df = live_troubleshoot_df.append(_live_troubleshoot_df, ignore_index=True)
    print(f"Batch of {batchOf} requests no. {str(i+1)}/{str(len([i for i in Chunks(live_universe, batchOf)]))} ended")

Batch of 100 requests no. 1/1 started
Batch of 100 requests no. 1/1 ended


In [20]:
live_troubleshoot_df.columns.name = live_inst
live_troubleshoot_df.describe(include='all')

2HSI16200L8.HF,Strike,MarketValueInDealCcy,RiskFreeRatePercent,UnderlyingPrice,PricingModelType,DividendType,UnderlyingTimeStamp,ReportCcy,VolatilityType,Volatility,DeltaPercent,GammaPercent,RhoPercent,ThetaPercent,VegaPercent,ErrorMessage
count,66.0,66.000000,66.000000,66.000000,66,66,66,66,66,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66
unique,NaN,NaN,NaN,NaN,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,BlackScholes,ImpliedYield,Default,HKD,Calculated,NaN,NaN,NaN,NaN,NaN,NaN,
freq,NaN,NaN,NaN,NaN,66,66,66,66,66,NaN,NaN,NaN,NaN,NaN,NaN,66
mean,16200.0,4469.651515,1.144962,17133.582727,NaN,NaN,NaN,NaN,NaN,37.893818,0.542227,0.000023,233.893921,-0.584083,119.829389,NaN
std,0.0,616.548644,0.057065,552.082884,NaN,NaN,NaN,NaN,NaN,2.715772,0.023745,0.000003,5.211079,0.043164,1.068325,NaN
min,16200.0,3441.000000,1.020000,16190.020000,NaN,NaN,NaN,NaN,NaN,33.562937,0.499498,0.000018,223.420010,-0.649037,117.660440,NaN
25%,16200.0,3804.250000,1.096250,16633.417500,NaN,NaN,NaN,NaN,NaN,34.762319,0.516847,0.000020,230.169372,-0.620582,119.012966,NaN
50%,16200.0,4677.500000,1.144000,17183.985000,NaN,NaN,NaN,NaN,NaN,39.386042,0.551180,0.000021,233.761419,-0.608863,119.852179,NaN
75%,16200.0,5004.750000,1.187750,17566.200000,NaN,NaN,NaN,NaN,NaN,40.047335,0.562027,0.000026,238.165069,-0.536224,120.628764,NaN


In [21]:
live_troubleshoot_df

2HSI16200L8.HF,Strike,MarketValueInDealCcy,RiskFreeRatePercent,UnderlyingPrice,PricingModelType,DividendType,UnderlyingTimeStamp,ReportCcy,VolatilityType,Volatility,DeltaPercent,GammaPercent,RhoPercent,ThetaPercent,VegaPercent,ErrorMessage
0,16200.0,4804.0,1.1460,17195.84,BlackScholes,ImpliedYield,Default,HKD,Calculated,40.463016,0.555145,0.000021,229.487303,-0.634743,118.871306,
1,16200.0,4772.0,1.1380,17213.87,BlackScholes,ImpliedYield,Default,HKD,Calculated,40.125391,0.554075,0.000021,230.628710,-0.627070,119.154242,
2,16200.0,4974.0,1.0950,17485.98,BlackScholes,ImpliedYield,Default,HKD,Calculated,40.628609,0.560791,0.000020,233.832127,-0.625678,120.005551,
3,16200.0,5016.0,1.0940,17517.40,BlackScholes,ImpliedYield,Default,HKD,Calculated,40.833513,0.562157,0.000020,233.810307,-0.628669,120.004036,
4,16200.0,5235.0,1.1715,17664.73,BlackScholes,ImpliedYield,Default,HKD,Calculated,41.815445,0.569821,0.000019,233.772260,-0.649037,119.741386,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,16200.0,3785.0,1.0880,16646.41,BlackScholes,ImpliedYield,Default,HKD,Calculated,34.509012,0.516294,0.000026,232.742035,-0.525111,119.787791,
62,16200.0,3770.0,1.0860,16645.98,BlackScholes,ImpliedYield,Default,HKD,Calculated,34.389513,0.515689,0.000026,232.969239,-0.522435,119.844663,
63,16200.0,3721.0,1.0380,16535.33,BlackScholes,ImpliedYield,Default,HKD,Calculated,34.547308,0.512554,0.000026,230.071212,-0.524968,119.349733,
64,16200.0,3475.0,1.0430,16224.45,BlackScholes,ImpliedYield,Default,HKD,Calculated,33.792644,0.500626,0.000027,224.898928,-0.517799,118.135055,


In [22]:
rd.close_session()

## Finding the objects to define your financial contract

Say you are looking for a way to define a Fixed Income instrument 'as of' a specific date; e.g.: AUSTRALIA, COMMONWEALTH OF (GOVERNMENT) bond `AU9YT=RR` priced with data from "2000-01-01". You could look into the [IPA Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-bond-contracts), and find `marketDataDate` under "Pricing Parameters"; you could then be tempted to use `pricing_parameters` in "Instrument Definition", but where is that exactly?

I thought it might be in `refinitiv.data.content.ipa.financial_contracts.bond.Definition`, so looked in there:

In [23]:
from refinitiv.data.content.ipa import financial_contracts as rdf
rdf.bond.Definition()

<refinitiv.data.content.ipa.financial_contracts.bond._definition.Definition object at 0x0000015D3E6F8DD0 {name='None'} {name='<refinitiv.data.content.ipa.financial_contracts.bond._bond_definition.BondInstrumentDefinition object at 0x0000015D3E7DC5D0>'}>

![pricingparams1](../Docs/pricingparams1.PNG)

You can see that the `pricing_parameters` variable takes `PricingParameters` objects... but what's that?

Turns out you can find out more with a Ctrl + click on `Definition` on Visual Studio:

![pricingparams2](../Docs/pricingparams2.png)

You will then be sent to the source code for `Definition`:

![pricingparams3](../Docs/pricingparams3.png)

What's interesting here is the path, highlighted in red. You can navigate through the `PricingParameters` (shown in line 4 in the screenshot):

![pricingparams4](../Docs/pricingparams4.png)

And you can now find the correct module for `PricingParameters`:

![pricingparams5](../Docs/pricingparams5.png)

In [24]:
rd.open_session()

definitions = [
    rdf.bond.Definition(
        instrument_code=i,
        pricing_parameters=rdf.bond._bond_pricing_parameters.PricingParameters(
            market_data_date="2000-01-01T00:00:00Z"))
    for i in ['AU9YT=RR', '225401BB3=2M', '61772BAC7=1M']]
response = rdf.Definitions(
    universe=definitions,
    fields=[
        'Isin',
        'RIC',
        'Cusip',
        'Sedol',
        'Ticker',
        'CashFlowDatesArray',
        'CashFlowInterestPercentsArray',
        'CashFlowCapitalAmountsInDealCcyArray',
        'CashFlowAnnualRatesPercentArray',
        'CashFlowTotalPercentsArray']).get_data()
response.data.df

,Isin,RIC,Cusip,Sedol,Ticker,CashFlowDatesArray,CashFlowInterestPercentsArray,CashFlowCapitalAmountsInDealCcyArray,CashFlowAnnualRatesPercentArray,CashFlowTotalPercentsArray
0,AU0000XCLUI7,AU9YT=RR,<NA>,6060019,AUGV,"[2000-02-15T00:00:00Z, 2000-08-15T00:00:00Z, 2...","[4.375, 4.375, 4.375, 4.375, 4.375, 4.375, 4.3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.75, 8.75, 8.75, 8.75, 8.75, 8.75, 8.75, 8.7...","[4.375, 4.375, 4.375, 4.375, 4.375, 4.375, 4.3..."
1,US225401BB38,225401BB3=2M,225401BB3,BMHDMY3,UBSG,<NA>,<NA>,<NA>,<NA>,<NA>
2,US61772BAC72,61772BAC7=1M,61772BAC7,BN7LY82,MS,<NA>,<NA>,<NA>,<NA>,<NA>
